In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import _init_paths

import os
import sys
import cv2
import json
import copy
import numpy as np
from opts import opts
from detector import Detector

output_video_path = 'output.mp4'
image_ext = ['jpg', 'jpeg', 'png', 'webp']
video_ext = ['mp4', 'mov', 'avi', 'mkv']
time_stats = ['tot', 'load', 'pre', 'net', 'dec', 'post', 'merge', 'display']
class_name = ['pedestrain', 'car','van','truck','bus']
obj_category = ['1', '4', '5', '6', '9']
class_colors = [(0, 0, 0), (0, 0, 255), (0, 255, 0), (255, 255, 0), (255, 0, 0)]

def convert_frames_to_video(frames, output_FPS):
  print("INSIDE")
  print(output_FPS)
  fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Be sure to use lower case
  height, width, channels = frames[0].shape
  out = cv2.VideoWriter(output_video_path, fourcc, output_FPS, (width, height))
  for i, ff in enumerate (frames):
      out.write(ff)
  out.release
  cv2.destroyAllWindows()
  return

/home/casper/anaconda3/envs/ct/lib/python3.6/site-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


In [2]:
def demo(opt):
  os.environ['CUDA_VISIBLE_DEVICES'] = "0"
  opt.debug = max(opt.debug, 1)
  detector = Detector(opt)

  if opt.demo == 'webcam' or \
    opt.demo[opt.demo.rfind('.') + 1:].lower() in video_ext:
    is_video = True
    # demo on video stream
    cam = cv2.VideoCapture(0 if opt.demo == 'webcam' else opt.demo)
  else:
    is_video = False
    # Demo on images sequences
    if os.path.isdir(opt.demo):
      image_names = []
      ls = os.listdir(opt.demo)
      for file_name in sorted(ls):
          ext = file_name[file_name.rfind('.') + 1:].lower()
          if ext in image_ext:
              image_names.append(os.path.join(opt.demo, file_name))
    else:
      image_names = [opt.demo]

  # Initialize output video
  out = None
  out_name = opt.demo[opt.demo.rfind('/') + 1:]
  print('out_name', out_name)
  if opt.save_video:
    # fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fourcc = cv2.VideoWriter_fourcc(*'H264')
    out = cv2.VideoWriter('../results/{}.mp4'.format(
      opt.exp_id + '_' + out_name),fourcc, opt.save_framerate, (
        opt.video_w, opt.video_h))
  
  if opt.debug < 5:
    detector.pause = False
  cnt = 0
  results = {}
  labelled = []

  while True:
      if is_video:
        _, img = cam.read()
        if img is None:
          save_and_exit(opt, out, results, out_name, )
      else:
        if cnt < len(image_names):
          img = cv2.imread(image_names[cnt])
        else:
          print("Going to enter:")
          convert_frames_to_video(labelled, 2)
          save_and_exit(opt, out, results, out_name)
      cnt += 1

      # resize the original video for saving video results
      if opt.resize_video:
        img = cv2.resize(img, (opt.video_w, opt.video_h))

      # skip the first X frames of the video
      if cnt < opt.skip_first:
        continue
      
      # cv2.imshow('input', img)

      # track or detect the image.
      ret = detector.run(img)

      # results[cnt] is a list of dicts:
      #  [{'bbox': [x1, y1, x2, y2], 'tracking_id': id, 'category_id': c, ...}]
      results[cnt] = ret['results']

      for j, ll in enumerate(ret['results']):
        # print(ll)
        score = ll['score']
        trd = ll['tracking_id']
        # print(type(score))
        bb = ll['bbox']
        # label = 'vehicle'
        #         print(bb[0])
        label = str(class_name[ll['class']-1])+str(trd)
        # print(label)
        p = (int(bb[0])-2, int(bb[1])-5)
        cv2.putText(img, label, p, cv2.FONT_HERSHEY_PLAIN, 1, class_colors[ll['class']-1], 1, cv2.LINE_AA)
        cv2.rectangle(img, (bb[0], bb[1]), (bb[2], bb[3]), class_colors[ll['class']-1], 1)
        labelled.append(img)
        # img2 = img
        # img2 = cv2.resize(img2, None, fx=0.75, fy=0.75, interpolation=cv2.INTER_CUBIC)
        # cv2.imshow("Image", img)
        # cv2.waitKey(0)

      # # save debug image to video
      # if opt.save_video:
      #   out.write(ret['generic'])
      #   if not is_video:
      #     cv2.imwrite('../results/demo{}.jpg'.format(cnt), ret['generic'])
      
      # esc to quit and finish saving video
      # if cv2.waitKey(1) == 27:
      #   save_and_exit(opt, out, results, out_name)
      #   return
  # save_and_exit(opt, out, results)

In [3]:
def save_and_exit(opt, out=None, results=None, out_name=''):
  if opt.save_results and (results is not None):
    save_dir =  '../results/{}_results.json'.format(opt.exp_id + '_' + out_name)
    print('saving results to', save_dir)
    json.dump(_to_list(copy.deepcopy(results)), 
              open(save_dir, 'w'))
  if opt.save_video and out is not None:
    out.release()
  sys.exit(0)

def _to_list(results):
  for img_id in results:
    for t in range(len(results[img_id])):
      for k in results[img_id][t]:
        if isinstance(results[img_id][t][k], (np.ndarray, np.float32)):
          results[img_id][t][k] = results[img_id][t][k].tolist()
  return results

if __name__ == '__main__':
  opt = opts().init()
  demo(opt)

usage: ipykernel_launcher.py [-h] [--dataset DATASET]
                             [--test_dataset TEST_DATASET] [--exp_id EXP_ID]
                             [--test] [--debug DEBUG] [--no_pause]
                             [--demo DEMO] [--load_model LOAD_MODEL]
                             [--resume] [--gpus GPUS]
                             [--num_workers NUM_WORKERS]
                             [--not_cuda_benchmark] [--seed SEED]
                             [--not_set_cuda_env] [--print_iter PRINT_ITER]
                             [--save_all] [--vis_thresh VIS_THRESH]
                             [--debugger_theme {white,black}] [--eval_val]
                             [--save_imgs SAVE_IMGS]
                             [--save_img_suffix SAVE_IMG_SUFFIX]
                             [--skip_first SKIP_FIRST] [--save_video]
                             [--save_framerate SAVE_FRAMERATE]
                             [--resize_video] [--video_h VIDEO_H]
                    

SystemExit: 2

/home/casper/anaconda3/envs/ct/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
